CONVULSE:  mean computer tells you to do things (╯°□°）╯︵ ┻━┻

Use your head (literally) to follow instructions. Emote went told, shift when told!
Task: Tilt to direction, Change Emotions, Sway

- Uses Webcam
- Detects Face
- Detects Facial Emotions with CNN (Happy, Sad, Angry, Neutral, Disgust, Fear, Surprise)
- Detects Motion (Swaying)
- Manipulates Webcam imagery (Video effects for added difficulty)

Inspiration: Bop-it but with your face


• Python + OpenCV image handling - webcam 
• Object detection (face, person, vehicle) - detects ur face 
• Object tracking and motion analysis - detects your action
• Image manipulation and processing - glitches your screen to make it harder
• Visualizing CNN features - Emotions

More than 3 computer vision techniques used +1 pt/
Real-time webcam input +1 pt /
Integration of CNN using TensorFlow/Keras +1 pt/
Project published on GitHub with README.md +1 pt /


In [1]:
# Libraries used
import tkinter as tk
from tkinter import messagebox
import cv2
import random
import time
from keras.models import load_model
import numpy as np


# Load CNN model
emotion_model = load_model("fer2013_mini_XCEPTION.102-0.66.hdf5", compile=False)
# Emotion labels from FER-2013
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']


# Start of Webcam Game Environment
def start_game():
    import random
    import time

    # tallies the number of fails in a row you can make before you are forced to exit
    fail_combo = 0

    # Load face cascade
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        messagebox.showerror("Error", "Webcam not found!")
        return

    # List of game commands
    commands = ["TILT RIGHT!","TILT LEFT!","SMILE!", "BE SAD!", "RAGE!!!", "LOCK IN.", "SWAY!"]
    current_command = ""
    command_to_emotion = {
    "SMILE!": "Happy",
    "BE SAD!": "Sad",
    "LOCK IN.": "Neutral",
    "RAGE!!!": "Angry"
    }
    last_command_time = time.time()
    command_interval = 2  # seconds
    score = 0
    command_check=""
    challenge_completed = False
    challenge_missed =  False
    face_positions = []  # Store X positions of face centers
    max_positions = 15   # How many positions to keep
    sway_threshold = 40  # Min movement in pixels to count as sway

    #Screen Effects
    last_effect_time = time.time()
    effect_interval = 5  # seconds between effects
    current_effect = None
    effect_duration = 2  # how long to keep it on
    while True:
        ret, frame = cap.read()
        
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        current_time = time.time()

        # Trigger a new effect every few seconds
        if current_time - last_effect_time > effect_interval+ effect_duration:
            current_effect = get_random_effect()  # Just reassign the function
            last_effect_time = current_time
        
        # Apply the effect if active
        if current_effect and current_time - last_effect_time < effect_duration:
            frame = current_effect(frame)

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_width = frame.shape[1]


        # Detect face
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        for (x, y, w, h) in faces:
            roi_gray = gray[y:y+h, x:x+w]
            roi_gray = cv2.resize(roi_gray, (64, 64))
            roi_gray = roi_gray.astype("float") / 255.0
            roi_gray = np.expand_dims(roi_gray, axis=-1)
            roi_gray = np.expand_dims(roi_gray, axis=0)

            preds = emotion_model.predict(roi_gray, verbose=0)[0]
            emotion_label = emotion_labels[preds.argmax()]
            face_center_x = x + w // 2

        
            # Draw emotion label
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, emotion_label, (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
   
            
            # Check if emotion matches command
            expected_emotion = command_to_emotion.get(current_command)
            if expected_emotion and emotion_label == expected_emotion and not challenge_completed:
                score += 1
                command_check = "Good job!"
                last_command_time = time.time()
                challenge_completed = True 
                challenge_missed = False
            
            if current_command == "TILT LEFT!" and face_center_x < frame_width * 0.3 and not challenge_completed:
                score += 1
                command_check = "WOWZERS!"
                challenge_completed = True
                last_command_time = time.time()
            
            elif current_command == "TILT RIGHT!" and face_center_x > frame_width * 0.7 and not challenge_completed:
                score += 1
                command_check = "SMASHING..!"
                challenge_completed = True
                last_command_time = time.time()

            if current_command == "SWAY!" and not challenge_completed:
                face_center_x = x + w // 2
                face_positions.append(face_center_x)
            
                if len(face_positions) > max_positions:
                    face_positions.pop(0)  # Keep buffer size small
            
                # Detect swaying: large enough left-right-left movement
                if len(face_positions) >= 5:
                    delta1 = face_positions[-1] - face_positions[-3]
                    delta2 = face_positions[-3] - face_positions[-5]
            
                    # If direction changed twice and moved far enough
                    if np.sign(delta1) != np.sign(delta2) and abs(delta1) > sway_threshold and abs(delta2) > sway_threshold:
                        score += 1
                        command_check = "MASTERFUL SWAYING!!"
                        challenge_completed = True
                        last_command_time = time.time()
                        face_positions.clear()


                
        # Update command every few seconds
        current_time = time.time()
        if current_time - last_command_time > command_interval:
            if not challenge_completed:
                fail_combo+= 1
                command_check = "FAILURE..."
                score -= 1
                challenge_missed = True
    
                if fail_combo >= 5:
                    break 
            else:
                fail_combo = 0
                challenge_missed = False            
            current_command = random.choice(commands)
            last_command_time = current_time
            challenge_completed = False
        
        # Display current command
        cv2.putText(frame, f"COMMAND: {current_command}", (30, 50),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 3)

        cv2.putText(frame, f"{command_check}", (30, 130),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 3)

        cv2.putText(frame, f"Score: {score}", (30, 90),
            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        if current_effect and current_time - last_effect_time < effect_duration:
            cv2.putText(frame, f"VISUAL CHAOS: {effect_name}", (30, frame.shape[0] - 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)


        cv2.line(frame, (int(frame_width * 0.3), 0), (int(frame_width * 0.3), frame.shape[0]), (0, 255, 0), 2)
        cv2.line(frame, (int(frame_width * 0.7), 0), (int(frame_width * 0.7), frame.shape[0]), (0, 255, 0), 2)


        cv2.imshow("CONVULSE: Game Window", frame)
        

        if cv2.waitKey(1) & 0xFF == 27:  # ESC
            break

    cap.release()
    cv2.destroyAllWindows()
    
    if fail_combo >= 5:
        messagebox.showinfo("Game Over", f"TOO MANY MISTAKES!! TERMINATE!! Final score: {score} (⊙x⊙;)")

def show_instructions():
    messagebox.showinfo("Instructions", 
        "Welcome to CONVULSE: mean computer tells you to do things (╯°□°）╯︵ ┻━┻\n\n"
        "- Press Start to begin.\n"
        "- Called Convulse because you will move around like you're convulsing \n"
        "- Use your head (literally) to follow instructions. Emote went told, shift when told!\n"
        "- Task: Tilt to direction, Change Emotions, Sway"
        "- Press ESC during the game to quit."
    )

# Gets a random effects to play every 5 seconds for 2 seconds, to mess
def get_random_effect():
    effect = random.choice(['invert', 'blur', 'rgb_shift', 'pixelate',  'flip'])
    global effect_name

    if effect == 'invert':
        effect_name = 'Invert Colors'
        return lambda frame: cv2.bitwise_not(frame)

    elif effect == 'blur':
        effect_name = 'Blur'
        return lambda frame: cv2.GaussianBlur(frame, (15, 15), 0)

    elif effect == 'rgb_shift':
        effect_name = 'RGB Shift'
        def rgb_shift(frame):
            b, g, r = cv2.split(frame)
            r = np.roll(r, 20, axis=1)
            g = np.roll(g, -15, axis=0)
            return cv2.merge([b, g, r])
        return rgb_shift

    elif effect == 'pixelate':
        effect_name = 'Pixelate'
        def pixelate(frame):
            small = cv2.resize(frame, (32, 24), interpolation=cv2.INTER_LINEAR)
            return cv2.resize(small, frame.shape[:2][::-1], interpolation=cv2.INTER_NEAREST)
        return pixelate

    elif effect == 'flip':
        flip_desc = {0: "Vertical Flip", 1: "Horizontal Flip", -1: "Mirror Flip"}
        flip_type = random.choice([-1, 0, 1])
        effect_name = flip_desc[flip_type]
        return lambda frame: cv2.flip(frame, flip_type)



# Setup basic Tkinter GUI
root = tk.Tk()
root.title("CONVULSE Launcher")
root.geometry("300x200")

title_label = tk.Label(root, text="ヽ（≧□≦）ノ CONVULSE ╰(艹皿艹 )", font=("Arial", 10))
title_label.pack(pady=10)

instructions_btn = tk.Button(root, text="Instructions", command=show_instructions)
instructions_btn.pack(pady=5)

start_btn = tk.Button(root, text="Start Game", command=start_game)
start_btn.pack(pady=5)

quit_btn = tk.Button(root, text="Quit", command=root.destroy)
quit_btn.pack(pady=5)

root.mainloop()
